# Udacity CarND Term 1 - Project 1 - Lane Finding
***
* ibalpowr@gmail.com
* Jan'17 cohort

In [ ]:
import os
files = os.listdir("test_images/")

### define converting function ... from rgb to gray

In [ ]:
import cv2
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

### set kernel size 3x3 for blur

In [ ]:
kernel_size = 3

### set up thresholds for canny

In [ ]:
high_threshold = 100
low_threshold = 50

### define canny edge

In [ ]:
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

### define polygon's vertices

In [ ]:
import numpy as np
vertices = np.array([[(30, 540), (450, 320), (500,320), (930,540)]],
                   dtype=np.int32)

### define Region of Interest (ROI) function

In [ ]:
def ROI (img, vertices):

    # 1st create a same size empty mask as the input image
    mask = np.zeros_like(img, dtype=img.dtype)
    
    # 2nd since the bitwise is an "and" operator,
    # the polygon color is white, i.e. all 1's
    polygon_color = 255
    
    # 3rd place polygon into the mask
    cv2.fillPoly(mask, vertices, polygon_color)
    
    # 4th bitwise image and the mask
    img_masked = cv2.bitwise_and(img, mask)
    
    return img_masked
    

### set up parameters for opencv2 hough

In [ ]:
# five hough transform parameters for cv2.HoughLinesP()

rho = 1 
theta = np.pi/180 
threshold = 50 
min_line_length = 1   
max_line_gap = 3000 

### define a lines_slopes function

In [ ]:
def find_lines_slopes(lines):
    slopes = []
    for i in range(len(lines)):
        slopes.append((lines[i,0,3]-lines[i,0,1])/
                      (lines[i,0,2]-lines[i,0,0] + 
                       np.random.normal(0,1e-9)))
    slopes_array=np.array(slopes).astype(np.float32)
    slopes_array=np.reshape(slopes_array, (slopes_array.shape[0],1))
    lines_array=np.reshape(lines,(lines.shape[0],4))
    lines_slopes = np.hstack([lines_array,slopes_array])
    lines_slopes = np.reshape(lines_slopes, (lines_slopes.shape[0],1,5))
    new_lines_slopes = lines_slopes[((lines_slopes[:,:,4] >= 0.5) 
                                 & (lines_slopes[:,:,4] <= 0.8)) | 
                                ((lines_slopes[:,:,4] >= -0.85) 
                                 & (lines_slopes[:,:,4] <= -0.55))]
    new_lines_slopes = np.reshape(new_lines_slopes, 
                                  (new_lines_slopes.shape[0],1,5))
    new_lines_slopes = new_lines_slopes.astype(np.uint16)
    return new_lines_slopes

### define a superimpose function

In [ ]:
def superimpose (lines, img):
    line_img = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2,slope in line:
            cv2.line(line_img, (x1, y1), (x2, y2), [255, 0, 0], 2)
    new_img = cv2.addWeighted(line_img, 0.4, img, 1, 0)
    return new_img

### process image file

In [ ]:
import matplotlib.image as mpimg
for i in range(len(files)):
    image = mpimg.imread('test_images/' + files[i])
    image_gray = grayscale(image)
    image_gray_3x3 = cv2.GaussianBlur(image_gray, 
                                    (kernel_size, kernel_size), 0)
    edges = canny(image_gray_3x3, low_threshold, high_threshold)
    edges_masked = ROI(edges, vertices)
    lines = cv2.HoughLinesP(edges_masked, rho, theta, threshold, 
                          np.array([]), min_line_length, max_line_gap)
    lines_slopes = find_lines_slopes(lines)
    new_image = superimpose(lines_slopes,image)
    mpimg.imsave('test_images/'+files[i]+'.png', new_image)

### process videos

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
import matplotlib.image as mpimg
def process_image(image):
    image_gray = grayscale(image)
    image_gray_3x3 = cv2.GaussianBlur(image_gray, 
                                    (kernel_size, kernel_size), 0)
    edges = canny(image_gray_3x3, low_threshold, high_threshold)
    edges_masked = ROI(edges, vertices)
    lines = cv2.HoughLinesP(edges_masked, rho, theta, threshold, 
                          np.array([]), min_line_length, max_line_gap)
    lines_slopes = find_lines_slopes(lines)
    new_image = superimpose(lines_slopes,image)
    return new_image

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
white_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
white_output = 'curve.mp4'
clip1 = VideoFileClip("challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)